## Single Head Initialization
In a Transformer, each token is passed through the model as a vector. The `hidden_size` defines how wide the token vector is when it reaches the Attention mechanism.

Our Attention layers will allow disabling bias terms for linear layers since recent papers and models, such as Cramming, Pythia, and PaLM, have shown that disabling the bias term results in little-to-no downstream performance drop while decreasing computational and memory requirements. However, to match the original Transformer implementation, we will set it on by default.

In this implementation, we will start merge Wa, WK 5, and WY into single linear layer, Wakv, and unbind the outputs into Q, K, and V. 

This is accomplished by increasing the output shape by a factor of three. This is mathematically equivalent to three individual linear layers, each with the same input and output shape.

In Multi-Head Attention, each individual head size is smaller than the input size?, so for Single Head we will arbitrarily set the head size to be four times smaller than the input dimension.

In [1]:
import numpy as np
import torch
from torch import nn

In [ ]:
class SingleHeadAttention(nn.Module):
    def __init__(self, hidden_size: int, bias: bool = True):
        
        